### Tutoriel des Données MET Phoenix

___
**Tutoriel** : Ce tutoriel fournit une introduction à la structure et aux cas d'utilisation des données de Pression et Température ainsi que des données Lidar Atmosphérique collectées par l'Instrument MET dans le cadre du voyage de l'Atterrisseur Phoenix vers Mars en 2008.

**Mission et Instrument** : [Phoenix Mars 2008 & Instrument MET](https://www.asc-csa.gc.ca/eng/astronomy/mars/phoenix/)

**Objectifs de la Mission** :
*Étudier l'habitabilité de l'environnement arctique martien*

**Données** : [Données de l'Instrument MET](https://donnees-data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/MET/)

**Exigences du système** : Python 3.12 ou ultérieur

**Niveau du tutoriel** : Intermédiaire

___
**Licence MIT**

Copyright (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence spatiale canadienne, 2025.

Droit d'auteur (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence Spatiale Canadienne, 2025.

Pour plus d'informations, veuillez vous référer au fichier License.txt.
___

#### Importation des Bibliothèques Requises

In [ ]:
# For data extraction
import os # for directory exploration
import requests # for data download through http requests
from bs4 import BeautifulSoup # for pulling data 
from itertools import product # for parsing through folders

# For data analysis & visualization
import pandas as pd # for processing
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting 

### 1. Introduction

##### Objectif du Tutoriel

Bienvenue dans le tutoriel des données MET Phoenix !

Dans ce tutoriel vous allez :
- Apprendre comment télécharger les données du site de Données Ouvertes de l'ASC
- Acquérir une compréhension du type de données qui ont été collectées
- Explorer des exemples de visualisations et d'analyses pour les données

##### Contexte de la Mission

L'Atterrisseur Phoenix a passé 5 mois dans la région arctique de Mars avec les objectifs scientifiques principaux d'explorer l'habitabilité, la géomorphologie, et le climat sur Mars. La composante canadienne de l'Atterrisseur était l'Instrument MET, qui suivait la météo à la surface et a notamment détecté la première instance de neige sur Mars.

#### 2. Comprendre les Données

##### Aperçu des Types de Données

L'Instrument MET avait deux composantes qui ont produit les données que nous utilisons dans ce tutoriel.

Une était un instrument Lidar vertical pour sonder l'atmosphère - il a collecté des diffusions atmosphériques entre 0 et 20 km avec un canal de comptage de photons de 532 nm (meilleur pour la détection de lumière de faible niveau), et des diffusions atmosphériques dans deux canaux analogiques - 532nm et 1064nm - entre 0 et 10 km (meilleur pour la détection de lumière de haut niveau).

La seconde était un mât avec trois capteurs de température à différentes hauteurs (250m, 500m, et 1000m) et un capteur de pression.

##### Structure et Format des Données

Avant de travailler réellement avec les données, entrons rapidement dans la structure elle-même !

Il y a deux "types" différents de données disponibles ici, divisés par les deux parties différentes de l'instrument MET. Le premier est Lidar, connu sous le nom de *"l", "ld", "LS"* dans le répertoire de dossiers des données. Le second est Pression / Température, connu sous le nom de *"pt", "mt", ou "PS"* dans le répertoire de dossiers des données.

Dans Lidar et Pression/Température, il existe un *"Enregistrement de Données Élémentaires"* (EDR) et un *"Enregistrement de Données Réduites"* (RDR).

L'EDR est essentiellement les données brutes, c'est exactement comme l'Instrument MET a communiqué avec l'ordinateur de l'Atterrisseur Phoenix, converti en ASCII et avec l'horodatage de l'atterrisseur. Ces données sont moins intuitives à travailler, et ne seraient généralement pas recommandées à moins que vous en sachiez plus sur la mission.

Le RDR est une version traitée des données originales, avec la durée de chaque mesure convertie de Compte de Trame en Secondes Terrestres et d'autres changements d'unité pour rendre les données plus faciles à travailler et plus compréhensibles.

Ce tutoriel se concentrera principalement sur le RDR.

Dans les deux Enregistrements de Données, il y a quelques catégories de plus avant d'arriver aux données elles-mêmes.

Voici un résumé :

***Ensembles de Données Lidar***

Dans l'Enregistrement de Données Élémentaires :
- Comptage de Photons (ELP)
- Comptage Analogique (ELA)
- Données Supplémentaires (ELS)

Dans l'Enregistrement de Données Réduites :
- Comptage de Photons (RLP)
- Comptage Analogique (RLA)
- Données Supplémentaires (RLS)

***Ensembles de Données Pression et Température***

Dans l'Enregistrement de Données Élémentaires :
- Basse Résolution (EML)
- Haute Résolution (EMH)

Dans l'Enregistrement de Données Réduites :
- Basse Résolution (RML)
- Haute Résolution (RMH)
- Pression Corrigée (RMC)
- Pression Auxiliaire (RMA)

Dans chacun de ces ensembles de données, il y a typiquement 150 fichiers CSV, représentant la prise de données sur les 150 jours solaires. Il y a occasionnellement des jours manquants dus à des problèmes avec l'Atterrisseur.

#### 3. Importation des Données

##### Utilisation de la fonction `download_met_data()`

Cette fonction vous permet de télécharger plusieurs fichiers à la fois selon diverses catégories dans le répertoire. Cela inclut :

- Pression et Température ou Lidar

      (`types` = `'pt'`ou `'l'`, ou `['l', 'RDR']` pour les deux)
- Jour(s) Solaire(s)

     (`days` = `'001'` à `'150'`, ou une gamme de jours `str(i).zfill(3) for i in range(1,150)` )
- Enregistrement de Données Élémentaires ou Enregistrement de Données Réduites

     (`records` = `'EDR'` ou `'RDR'`, ou `['EDR', 'RDR']` pour les deux)

Pour l'utiliser, vous devez d'abord définir un dossier dans lequel vous voulez télécharger les données sur votre ordinateur - ce sera là où vous accédez aux ensembles de données donc assurez-vous que c'est quelque chose que vous pouvez utiliser avec votre IDE.

Ensuite, vous pouvez appeler la fonction avec vos paramètres désirés, en utilisant ce qui précède comme guide. Il y a aussi quelques exemples ci-dessous.

In [ ]:
# set a folder you want to store all your files in
download_folder = "C:/Users/zaraya/Downloads/test_folder"

In [ ]:
# this function allows us to download the files needed based on different categories

# the parameters groups, sets, and cats are required due to the directory structure. 
# the default arguments parse through all 150 days (they are split into 3 sets), and lidar & pt (groups & cats)
def download_met_data(types, days, records, local_directory, groups = ['ld', 'mt'], sets = ['1', '2', '3'], cats = ['LS', 'MS']):

    # this part allows us to select based on categories we want 
    for type, group, set, cat, day, record in product(types, groups, sets, cats, days, records):
        # base url - where the data is hosted (csa open data page)
        base_url = f'https://donnees-data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/MET/phx-m-met-2-{type}-edr-v10/ph{group}_000{set}/DATA/{cat}{day}/{record}/'
        response = requests.get(base_url)
        soup = BeautifulSoup(response.text, "html.parser")

        for link in soup.find_all("a"):
            file_name = link.get("href")

            # downloads only CSV files
            if file_name.endswith(".csv"):
                file_url = base_url + file_name
                file_path = os.path.join(local_directory, file_name)
                
                with requests.get(file_url, stream=True) as r:
                    r.raise_for_status()
                    with open(file_path, "wb") as f:
                        for chunk in r.iter_content(chunk_size = 8192):
                            f.write(chunk)
                            
                print(f'Downloaded {file_name}')

##### Exemples de Chargement de Données
Voici quelques exemples pour charger différents groupes d'ensembles de données.

Si vous trouvez que votre code fonctionne lentement, vous pouvez changer certains des arguments par défaut dans les paramètres.

`groups` et `cats` représentent tous deux la même chose que `types` (lidar - `groups = 'ld'`, `cats = 'LS'` vs pression / température `groups = 'mt'`, `cats = 'MS'`) Donc si vous téléchargez seulement un type, vous pouvez le changer en conséquence

`sets` sont là parce que la structure de dossiers divise les 150 jours solaires en 3 ensembles, `sets = '1'` contenant les 30 premiers jours, ensemble `sets = '2'` étant les premiers 30 - 90, et ensemble `sets = '3'` étant les premiers 90 - 150. Si vous savez que votre gamme de dates appartient seulement à un ensemble spécifique ou une gamme d'ensembles, vous pouvez le spécifier.

In [ ]:
# if you want to download all files 

download_met_data(types = ['l','pt'],
                    days = [str(i).zfill(3) for i in range(1,150)],
                    records = ['EDR','RDR'], local_directory = download_folder)

In [ ]:
# downloading only the EDR Lidar datasets for the 61st solar day
download_met_data(types = ['l'],
                    days = ['061'],
                    records = ['EDR'], local_directory = download_folder)

# changing the defaults to have it run faster 
download_met_data(types = ['l'],
                    days = ['061'],
                    records = ['EDR'],
                      local_directory = download_folder,
                      groups = 'ld', sets= '2', cats = 'LS')

Dans ce tutoriel, je vais utiliser seulement les données du SOL65 (Jour solaire 65) et de l'Enregistrement de Données Réduites, mais à la fois les données Lidar et Pression et Température

In [ ]:
download_met_data(types = ['l', 'pt'],
                    days = ['065'],
                    records = ['RDR'], local_directory = download_folder)

#### 4. Explorer le Lidar

L'enregistrement de données réduites contient 3 ensembles de données différents, tous avec des temps convertis en secondes terrestres, et les gammes d'altitude en mètres

- RLA est Données Analogiques Lidar RDR - contient l'intensité moyenne globale de rétrodiffusion aux longueurs d'onde de 532nm et 1064nm

- RLP est Données de Comptage de Photons Lidar RDR - contient le nombre total de photons à la longueur d'onde de 532 nm

- RLS est Données Lidar Supplémentaires - contient des informations supplémentaires : moyenne, min et max de l'intensité de rétrodiffusion laser et analogique, et température du châssis laser.

Dans ce tutoriel nous nous concentrerons sur RLA et RLP.

Pour plus d'informations telles que les heures de début, noms de colonnes et unités, vous pouvez accéder aux fichiers d'étiquettes (LBL) du répertoire.

In [ ]:
# import both RLA (analog) & RLP (photon) as data frames 

analog = pd.read_csv('Downloads/test_folder/LS065RLA_00901961095_1760M1.TAB.csv', header=None)
photon = pd.read_csv('Downloads/test_folder/LS065RLP_00901961095_1760M1.TAB.csv', header=None)
#supplementary = pd.read_csv('Downloads/test_folder/LS065RLS_00901961095_1760M1.TAB.csv', header=None) 

# adding column names for clarity
analog.columns = ['time', 'range', 'backscatter_intensity_532NM', 'backscatter_intensity_1064NM']
photon.columns = ['time', 'range', 'total_photon_counts_532NM']

In [ ]:
analog.describe()

In [ ]:
# note that the spatial range of photon counts is more sparse
photon.describe()

##### Analyse des Profils de Rétrodiffusion Atmosphérique

Voici un exemple d'un graphique exploratoire simple que nous pouvons faire en utilisant les données de comptage analogique et de photons.
Puisqu'il nous manque quelques jours à cause d'erreurs, nous utiliserons `lineplot()` de seaborn pour extrapoler certaines de ces valeurs manquantes.

In [ ]:
plt.figure(figsize = (5, 5))
sns.lineplot(x = analog['backscatter_intensity_532NM'], y = analog['range'])
plt.xlabel('Backscatter Intensity (V)')
plt.ylabel('Height (m)')
plt.title('Lidar analog backscatter signal by altitude at wavelength of 532NM')
plt.show()

In [ ]:
# and for the 1064 nm wavelength
plt.figure(figsize = (3, 5))
sns.lineplot(x = analog['backscatter_intensity_1064NM'], y = analog['range'])
plt.xlabel('Backscatter Intensity (V)')
plt.ylabel('Height (m)')
plt.title('Lidar analog backscatter signal by altitude at wavelength of 1064NM')
plt.show()

In [ ]:
# and now for the photon counts 

plt.figure(figsize = (3, 5))
sns.lineplot(x = photon['total_photon_counts_532NM'], y = photon['range']/1000)
plt.xlabel('Photon Counts (Hz)')
plt.ylabel('Height (km)')
plt.title('Lidar total photon counts by altitude at wavelength of 532NM')
plt.show()

Le Comptage Analogique est typiquement utilisé pour des gammes d'altitude plus courtes que ce qui a été enregistré, donc il peut aussi être utile de regarder l'intensité de rétrodiffusion comparée à des gammes plus courtes.

In [ ]:
plt.figure(figsize = (3, 5))
sns.lineplot(x = analog.query('range < 2500')['backscatter_intensity_532NM'], y = analog.query('range < 2500')['range'])
plt.xlabel('Backscatter Intensity (V)')
plt.ylabel('Height (m)')
plt.title('Lidar analog backscatter signal (less than 2.5km)')
plt.show()

##### 5. Explorer la Pression / Température

Similairement aux données Lidar, je vais travailler avec les données réduites seulement pour ce tutoriel.

L'enregistrement de données réduites pour P/T contient 4 ensembles de données différents, tous avec des temps convertis en secondes terrestres, et les gammes d'altitude en mètres

- RMA est Données Auxiliaires RDR - il contient des données de pression et température non corrigées et en tant que tel n'est pas le meilleur pour l'analyse, connu principalement comme données de "maintenance", il est gardé pour la tenue de dossiers.

- RMC est Données Corrigées RDR - il contient des valeurs de pression corrigées, qui tiennent compte de la chaleur rayonnée par l'instrument lui-même

- RML sont des données RDR Basse Résolution - contient la moyenne, écarts types, min et max des températures à 3 altitudes (250m, 500m, et 1000m) toutes les 512 secondes terrestres.

- RMH sont des données RDR Haute Résolution - les entrées sont collectées soit par un mode de moyenne (signifiant soit un événement de température d'un changement de 15 Kelvin sur 512 secondes ou un événement de pression d'une différence de pression de 1 Pascal sur 512 secondes) ou quand les valeurs min et max de température à un moment dépassent le 'déclencheur' alias un seuil pré-établi.

Dans ce tutoriel nous nous concentrerons sur le RMC.

Pour plus d'informations telles que les heures de début, noms de colonnes et unités, vous pouvez accéder aux fichiers d'étiquettes (LBL) du répertoire.

In [ ]:
corrected = pd.read_csv('Downloads/test_folder/MS065RMC_00901976517_1761M1.TAB.csv', header=None)
highmet = pd.read_csv('Downloads/test_folder/MS065RMH_00901976517_1761M1.TAB.csv', header=None)
lowmet = pd.read_csv('Downloads/test_folder/MS065RML_00901976517_1761M1.TAB.csv', header=None)

corrected.columns = ['Time', 'Pressure', '250_Temperature', '500_Temperature', '1000_Temperature', 'Ref_Temperature']
lowmet.columns = ["Time", "Pressure_Avg", "Pressure_Stdev", "Pressure_Min", "Pressure_Max",
                   "250_Temperature_Avg", "250_Temperature_Stdev", "250_Temperature_Min", "250_Temperature_Max",
                     "500_Temperature_Avg", "500_Temperature_Stdev", "500_Temperature_Min", "500_Temperature_Max",
                     "1000_Temperature_Avg", "1000_Temperature_Stdev", "1000_Temperature_Min", "1000_Temperature_Max", 
                     "Ref_Temperature_Avg", "Ref_Temperature_Stdev", "Ref_Temperature_Min", "Ref_Temperature_Max",
                     "Trigger_Sensor"]
highmet.columns = ["Time", "Pressure", "250_Temperature", "500_Temperature", "1000_Temperature", "Reference_Temperature"]

##### Variations de Température à Différentes Altitudes

L'Instrument MET avait un mât à trois hauteurs différentes (250mm, 500mm, et 1000mm) et mesurait la température environnante à chaque hauteur.
Voici un exemple de visualisation pour comparer ces trois hauteurs, ainsi que la température de référence

In [ ]:
# let's look at  temperature variations at the different altitudes 
plt.figure(figsize = (10, 6))
plt.plot(corrected['Time'], corrected['250_Temperature'], label='250 mm')
plt.plot(corrected['Time'], corrected['500_Temperature'], label='500 mm')
plt.plot(corrected['Time'], corrected['1000_Temperature'], label='1000 mm')
plt.plot(corrected['Time'], corrected['Ref_Temperature'], label='Absolute')

plt.legend()
plt.title('Exploration of Temperature Ranges across Time')
plt.xlabel('Time Elapsed since Start Time (s)')
plt.ylabel('Temperature (Kelvin)')
plt.margins(x=0)
plt.show()

##### Relation entre la pression et la température à différentes hauteurs

La pression a également été enregistrée par cet instrument, donc ci-dessous est une visualisation explorant la relation entre elle et les températures enregistrées. Voyez si vous pouvez observer une tendance générale.

In [ ]:
# correlation between pressure & temperature 

pressure_over_0 = corrected.query('Pressure > 0')['Pressure']

plt.figure(figsize = (10, 6))
sns.scatterplot(x = pressure_over_0,  y = corrected['250_Temperature'], label='250 mm')
sns.scatterplot(x = pressure_over_0, y = corrected['500_Temperature'], label='500 mm')
sns.scatterplot(x = pressure_over_0, y = corrected['1000_Temperature'], label='1000 mm')
sns.scatterplot(x = pressure_over_0, y = corrected['Ref_Temperature'], label='Absolute')

plt.legend()
plt.title('Pressure vs. Temperature Ranges')
plt.xlabel('Pressure (Pa)')
plt.ylabel('Temperature (Kelvin)')
plt.show()